 Автор: Лейман М.А.  
 Дата создания: 26.06.2025  

# Итоговая аттестация. 

Ссылка на gradio - https://huggingface.co/spaces/makc-mon173/image-segmentation

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import gradio as gr

In [ ]:
# Палитра и имена классов
palette = [
    [0, 0, 0],      # ФОН
    [128, 64,128],  # road
    [244, 35,232],  # sidewalk
    [70, 70, 70],   # building
    [102,102,156],  # wall
    [190,153,153],  # fence
    [153,153,153],  # pole
    [250,170, 30],  # traffic light
    [220,220,  0],  # traffic sign
    [107,142, 35],  # vegetation
    [152,251,152],  # terrain
    [70,130,180],   # sky
    [220, 20, 60],  # person
    [255,  0,  0],  # rider
    [0,  0,142],    # car
    [0,  0, 70],    # truck
    [0, 60,100],    # bus
    [0, 80,100],    # train
    [0,  0,230],    # motorcycle
    [119, 11, 32],  # bicycle
]

class_names = [
    "ФОН", "road", "sidewalk", "building", "wall", "fence", "pole",
    "traffic light", "traffic sign", "vegetation", "terrain", "sky",
    "person", "rider", "car", "truck", "bus", "train", "motorcycle", "bicycle"
]



session = ort.InferenceSession("files/unetpp_model.onnx")

def preprocess_image(img, size=(512, 256)):
    img = img.convert("RGB").resize(size)  # PIL: (width, height)
    img_np = np.array(img).astype(np.float32) / 255.0
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    img_np = (img_np - mean) / std
    img_np = img_np.transpose(2, 0, 1)  # HWC -> CHW
    img_np = np.expand_dims(img_np, axis=0).astype(np.float32)
    return img_np




def segment_and_overlay(input_img, selected_classes):
    orig_width, orig_height = input_img.size  # оригинальный размер

    # Предобработка и инференс (модель ожидает 256x512)
    input_tensor = preprocess_image(input_img) 
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})
    pred_mask = np.argmax(outputs[0], axis=1)[0]  # размер (256, 512)

    # Палитра и имена классов
    palette = [
        [0, 0, 0],      # ФОН
        [128, 64,128],  # road
        [244, 35,232],  # sidewalk
        [70, 70, 70],   # building
        [102,102,156],  # wall
        [190,153,153],  # fence
        [153,153,153],  # pole
        [250,170, 30],  # traffic light
        [220,220,  0],  # traffic sign
        [107,142, 35],  # vegetation
        [152,251,152],  # terrain
        [70,130,180],   # sky
        [220, 20, 60],  # person
        [255,  0,  0],  # rider
        [0,  0,142],    # car
        [0,  0, 70],    # truck
        [0, 60,100],    # bus
        [0, 80,100],    # train
        [0,  0,230],    # motorcycle
        [119, 11, 32],  # bicycle
    ]

    class_names = [
        "ФОН", "road", "sidewalk", "building", "wall", "fence", "pole",
        "traffic light", "traffic sign", "vegetation", "terrain", "sky",
        "person", "rider", "car", "truck", "bus", "train", "motorcycle", "bicycle"
    ]

    selected_indices = [class_names.index(cls) for cls in selected_classes]

    color_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)
    for cls_idx in selected_indices:
        color_mask[pred_mask == cls_idx] = palette[cls_idx]

    mask_img = Image.fromarray(color_mask)

    # Растягиваем маску до оригинального размера изображения
    mask_img = mask_img.resize((orig_width, orig_height), resample=Image.NEAREST)

    # Наложение маски на исходное изображение с прозрачностью
    base = input_img.convert("RGBA")
    overlay = mask_img.convert("RGBA")
    blended = Image.blend(base, overlay, alpha=0.5)

    return blended


In [ ]:

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1):
            selected = gr.CheckboxGroup(label="Выберите классы для отображения", choices=class_names, value=class_names[:3])
            btn = gr.Button("Сегментировать")
        with gr.Column(scale=3):
            input_image = gr.Image(type="pil", value="files/999.png", label="Исходное изображение")
            output_image = gr.Image(label="Результат сегментации")

    btn.click(fn=segment_and_overlay, inputs=[input_image, selected], outputs=output_image)

demo.launch()


/home/maksim/anaconda3/envs/myenv3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
